In [1]:
import pandas as pd
import json
import os
import json_repair
from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import WikipediaLoader
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate)
from langchain import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema import (SystemMessage,HumanMessage,AIMessage)
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_community.graphs.graph_document import Node, Relationship, GraphDocument

In [219]:
os.environ["GROQ_API_KEY"] = "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" # Provide GROQ API Key

# Neo4j
neo4j_url = "neo4j+s://f95622f0.databases.neo4j.io" # Provide neo4j link
neo4j_user = "neo4j" # Provide neo4j user
neo4j_password = "n9YghKN17YtYUnv5zpauZpkC1Z6ScAwK1JXugPDKTy0" # Provide neo4j Password
graph = Neo4jGraph(neo4j_url,neo4j_user,neo4j_password)

from neo4j import GraphDatabase

driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader("data.txt", encoding='utf8')
documents = loader.load()
print(documents)

[Document(metadata={'source': 'data.txt'}, page_content='"What is 5G ? 5G is the 5th generation mobile network which is the latest in mobile telecommunication technology. Jio is deploying the most advanced version of standalone 5G, which has zero dependencies on the existing 4G network. Other operators are deploying a non-standalone version of 5G that operates on their 4G core, making Jio True 5G far superior With standalone 5G, Jio can deliver new and powerful services such as low latency connectivity, massive machine-to-machine communication, 5G calls, edge computing, network slicing, and the metaverse.",\n"Why should I switch from 4G to 5G? India\'s only True 5G network provides an unparalleled combination of coverage, capacity, quality, and affordability compared to other networks. Here’s how 5G will revolutionise your experience: Standalone 5G architecture: World\'s most advanced 5G network with zero dependencies on 4G network Low latency, machine-to-machine communication, 5G call

In [5]:
docs = " ".join([d.page_content for d in documents]).replace("\n", "").replace("==", "")
print(docs)

"What is 5G ? 5G is the 5th generation mobile network which is the latest in mobile telecommunication technology. Jio is deploying the most advanced version of standalone 5G, which has zero dependencies on the existing 4G network. Other operators are deploying a non-standalone version of 5G that operates on their 4G core, making Jio True 5G far superior With standalone 5G, Jio can deliver new and powerful services such as low latency connectivity, massive machine-to-machine communication, 5G calls, edge computing, network slicing, and the metaverse.","Why should I switch from 4G to 5G? India's only True 5G network provides an unparalleled combination of coverage, capacity, quality, and affordability compared to other networks. Here’s how 5G will revolutionise your experience: Standalone 5G architecture: World's most advanced 5G network with zero dependencies on 4G network Low latency, machine-to-machine communication, 5G calls, edge computing, and network slicing Far superior to 4G-bas

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=30
)
split_docs = text_splitter.create_documents([docs])
print("Number of text docs made : ", len(split_docs))

[Document(page_content='"What is 5G ? 5G is the 5th generation mobile network which is the latest in mobile telecommunication technology. Jio is deploying the most advanced version of standalone 5G, which has zero dependencies on the existing 4G network. Other operators are deploying a non-standalone version of 5G that operates on their 4G core, making Jio True 5G far superior With standalone 5G, Jio can deliver new and powerful services such as low latency connectivity, massive machine-to-machine communication, 5G calls, edge computing, network slicing, and the metaverse.","Why should I switch from 4G to 5G? India\'s only True 5G network provides an unparalleled combination of coverage, capacity, quality, and affordability compared to other networks. Here’s how 5G will revolutionise your experience: Standalone 5G architecture: World\'s most advanced 5G network with zero dependencies on 4G network Low latency, machine-to-machine communication, 5G calls, edge computing, and network slic

In [ ]:
entity_types = ['technology','company','product','location','store','cost','service','offer','plan','customer','time','date','data']
relation_types = ['isProductOf','Compatible','isAvailableAt','LocatedIn','OfferedBy','RelatedTo','isCostOf','Provides','isNotEligibleFor','isEligibleFor','isBenefitOf','isValidityOf','IsPartOf']

In [ ]:
################### MODEL FOR TEXT SUMMARIZATION ######################

system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for converting text data into clear statements.
    Convert all information in text data into clear statements with relationships between entities and their description so that
    it can be used for knowledge graph making.
    Make statments and paragraphs. Don't make points and headings. Use entities in each statement. Connect all entities through statements.
    Give heading to each statement to show which entities it belongs to.
    Also state connection between the headings at the end.
    Don't leave out any information.
    """,
    input_variables = ["relationship_types"],
)


system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)


parser = JsonOutputParser()

human_prompt = PromptTemplate(
    template = """ For the folowing text summarize it \nText: {text}""",
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt_summarize = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatGroq(temperature=0, model_name="llama3-70b-8192")
chain_summarize = chat_prompt_summarize | model

In [10]:
summary = []
for doc in split_docs:
  response = chain_summarize.invoke({"relationship_types": relation_types, "text" : doc.page_content})
  summary.append(response.content)

In [13]:
for i in range(len(summary)):
  print(summary[i])

Here is the summarized text in clear statements with relationships between entities and their descriptions:

**What is 5G?**
5G is the 5th generation mobile network, which is the latest in mobile telecommunication technology.

**Jio's 5G Network**
Jio is deploying the most advanced version of standalone 5G, which has zero dependencies on the existing 4G network. Jio's standalone 5G can deliver new and powerful services such as low latency connectivity, massive machine-to-machine communication, 5G calls, edge computing, network slicing, and the metaverse.

**Jio True 5G vs Other Operators**
Jio True 5G is far superior to other operators' non-standalone 5G networks that operate on their 4G core.

**Why Switch to 5G?**
India's only True 5G network, Jio, provides an unparalleled combination of coverage, capacity, quality, and affordability compared to other networks.

**Jio True 5G Features**
Jio True 5G has a standalone 5G architecture, which is the world's most advanced 5G network with z

In [ ]:
################ MODEL FOR KG FORMATION #####################

system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
    Your task is to identify the entities and relations from a given text.
    The text will contain main entities and relationships between them, use these to find the exact ones.
    You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "head_description", "relation", "tail", "tail_type" and "tail_description".
    The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt.
    The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types}.4
    The "head_description" key must contain the description of the extracted head entity.
    The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types}.
    The "tail" key must represent the text of an extracted entity which is the tail of the relation.
    The "tail_type" key must contain the type of the tail entity from {entity_types}.
    The "tail_description" key must contain the description of the extracted tail entity.
    If any relation or entity doesn't fit within the types of relation and entities given then you are free to make a new relation or entity type.
    Extract as many relations and entities as you can whilst keeping most information in description.
    Don't miss information from any sentence.
    Look at the whole text when creating description for entities so as to not miss any information.
    Try to make both sided relationships between two entities
    Don't print any "text" key
    

    IMPORTANT NOTES:
    - Don't add any explanation and text.
    - ALL KEYS head, head_type, head_description, relation, tail, tail_type, tail_description SHOULD ALWAYS BE PRESENT
    """,
    input_variables=["entity_types","relation_types"],
)


system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)

examples = [
        {
            "text":"The ₹259 Prepaid Plan is a one-of-a-kind, ‘calendar-month validity’ plan from Jio." ,
            "head": "₹259 Prepaid Plan",
            "head_type": "plan",
            "head_description": "It is one-of-a-kind, ‘calendar-month validity’ plan for prepaid services",
            "relation": "isProductOf",
            "tail": "Jio",
            "tail_type": "company",
            "tail_description": "",
        },
        {
            "text":"5G Data Voucher provides benefit of 1 GB Data at the cost of Rs.15 with base plan validity" ,
            "head": "5G Data Voucher",
            "head_type": "plan",
            "head_description" : "",
            "relation": "BenefitOf",
            "tail": "1 GB Data",
            "tail_type": "data",
            "tail_description": "It costs Rs.15 and has validity of base plan",
        },
        {
            "text":"5G services can be accessed from any place in Mumbai, Maharashtra" ,
            "head": "5G services",
            "head_type": "technology",
            "head_description" : "",
            "relation": "isAvailableAt",
            "tail": "Mumbai",
            "tail_type": "location",
            "tail_description": "It is a city in Maharashtra",
        },
    ]

class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like 5G, Jio, Reliance Digital Store")
    head_type: str = Field(description="type of the extracted head entity like store, company, etc")
    head_description: str = Field(description="a few lines description of head entity")
    relation: str = Field(description="relation between the head and the tail entities")
    tail: str = Field(description="extracted second or tail entity like 5G, Jio, Reliance Digital Store")
    tail_type: str = Field(description="type of the extracted tail entity like store, company, etc")
    tail_description: str = Field(description="a few lines description of tail entity")

parser = JsonOutputParser(pydantic_object=ExtractedInfo)

human_prompt = PromptTemplate(
    template = """ Based on the following example, extract entities and relations from the provided text.\n\n

    Use the following entity types:
    # ENTITY TYPES:
    {entity_types}

    Use the following relation types:
    # RELATION TYPES:
    {relation_types}

    Below are a number of examples of text and their extracted entities and relationshhips.
    {examples}

    For the following text, generate extract entitites and relations. \n{format_instructions}\nText: {text}""",
    input_variables=["entity_types","relation_types","examples","text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = chat_prompt | model

In [21]:
graph_docs = []
responses = []
for doc in summary[84:]: # only 3 docs
    response  = chain.invoke({"entity_types" : entity_types, "relation_types" : relation_types, "examples" : examples, "text" : doc})
    relationships = []
    for rel in json_repair.loads(response.content):
        source_node = Node(id=rel["head"], type=rel["head_type"])
        target_node = Node(id=rel["tail"], type=rel["tail_type"])
        relationships.append(Relationship(source=source_node, target=target_node, type=rel['relation']))

    graph_document = GraphDocument(
        nodes=[],
        relationships=relationships,
        source={"text" : doc, "page_content": ""}
    )
    graph_docs.append(graph_document)
    responses.append(response.content)

In [17]:
for i in range(len(responses)):
  print(responses[i])

Here is the extracted entities and relations in the required JSON format:

[{"head": "5G", "head_type": "technology", "head_description": "It is the 5th generation mobile network, which is the latest in mobile telecommunication technology.", "relation": "isProductOf", "tail": "Jio", "tail_type": "company", "tail_description": ""},

{"head": "Jio", "head_type": "company", "head_description": "It is deploying the most advanced version of standalone 5G, which has zero dependencies on the existing 4G network.", "relation": "isAvailableAt", "tail": "India", "tail_type": "location", "tail_description": ""},

{"head": "Jio True 5G", "head_type": "product", "head_description": "It is the only True 5G network in India, providing an unparalleled combination of coverage, capacity, quality, and affordability compared to other networks.", "relation": "isProductOf", "tail": "Jio", "tail_type": "company", "tail_description": ""},

{"head": "Jio True 5G", "head_type": "product", "head_description": "I

In [9]:
################# MODEL FOR VECTOR EMBEDDING #####################

import re
from sentence_transformers import SentenceTransformer
encode_model = SentenceTransformer('all-MiniLM-L6-v2')

def add_spaces_to_camel_case(text):
    return re.sub(r'(?<!^)(?=[A-Z])', ' ', text)

c:\Users\poops\Desktop\Coding Shenanigans\SOIEvent\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\poops\Desktop\Coding Shenanigans\SOIEvent\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
################# NEO4J KG FORMATION #####################

def create_nodes_and_relationship(tx, head, head_type, head_description, tail, tail_type, tail_description, relation):
    # Check if a node with the same name but different type exists for head
    head_node = tx.run("""
    MATCH (n:Entity {name: $head})
    RETURN n.type AS type
    """, head=head).single()
    
    if head_node and head_node["type"] != head_type:
        head = f"{head} ({head_type})"
    
    # Check if a node with the same name but different type exists for tail
    tail_node = tx.run("""
    MATCH (n:Entity {name: $tail})
    RETURN n.type AS type
    """, tail=tail).single()
    
    if tail_node and tail_node["type"] != tail_type:
        tail = f"{tail} ({tail_type})"
    
    # Generate embeddings for head and tail
    head_vector = encode_model.encode(head + " " + head_description).tolist()
    head_name_vector = encode_model.encode(head).tolist()
    tail_vector = encode_model.encode(tail + " " + tail_description).tolist()
    tail_name_vector = encode_model.encode(tail).tolist()
    relationship_vector = encode_model.encode(add_spaces_to_camel_case(relation)).tolist()
    
    # Merge head node with conditional concatenation of description and add vector
    tx.run("""
    MERGE (h:Entity {name: $head, type: $head_type})
    ON CREATE SET h.description = $head_description, h.vector = $head_vector, h.namevector = $head_name_vector
    ON MATCH SET h.description = CASE
        WHEN h.description CONTAINS $head_description THEN h.description
        ELSE h.description + ' ' + $head_description
    END
    """, head=head, head_type=head_type, head_description=head_description, head_vector=head_vector, head_name_vector = head_name_vector)

    # Merge tail node with conditional concatenation of description and add vector
    tx.run("""
    MERGE (t:Entity {name: $tail, type: $tail_type})
    ON CREATE SET t.description = $tail_description, t.vector = $tail_vector, t.namevector = $tail_name_vector
    ON MATCH SET t.description = CASE
        WHEN t.description CONTAINS $tail_description THEN t.description
        ELSE t.description + ' ' + $tail_description
    END
    """, tail=tail, tail_type=tail_type, tail_description=tail_description, tail_vector=tail_vector, tail_name_vector = tail_name_vector)

    # Create relationship
    relationship_query = f"""
    MATCH (h:Entity {{name: $head}}), (t:Entity {{name: $tail}})
    MERGE (h)-[r:`{relation}`]->(t)
    ON CREATE SET r.vector = $relationship_vector
    ON MATCH SET r.vector = $relationship_vector
    """
    tx.run(relationship_query, head=head, tail=tail, relationship_vector=relationship_vector)

with driver.session() as session:
    for response in responses:
      for record in json_repair.loads(response):
        if all(key in record for key in ['head', 'head_type', 'tail', 'tail_type', 'tail_description', 'relation']):
            head = record['head']
            head_type = record['head_type']
            head_description = record['head_description']
            tail = record['tail']
            tail_type = record['tail_type']
            tail_description = record['tail_description']
            relation = record['relation']

            session.write_transaction(create_nodes_and_relationship, head, head_type, head_description, tail, tail_type, tail_description, relation)
        else:
           continue

C:\Users\poops\AppData\Local\Temp\ipykernel_2692\2186343321.py:67: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_and_relationship, head, head_type, head_description, tail, tail_type, tail_description, relation)


In [1]:
# with open('responses.txt', 'w', encoding='utf-8') as file:
#     # Write the string to the file
#     for response in responses:
#         # Write the string to the file followed by a newline character
#         file.write(response + '\n')

In [2]:
# for index, responser in enumerate(responses):
#     if responser == response:
#         print(f"The index of '{response}' is {index}.")
#         break

In [3]:
# def query_vector_search(question_vector, top_n=5):
#     with driver.session() as session:
#         query = """
#         WITH $question_vector AS questionVector
#         MATCH (n)
#         WHERE n.vector IS NOT NULL
#         WITH n, questionVector, n.vector AS nodeVector
#         WITH n, questionVector, nodeVector,
#             reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i] * nodeVector[i]) AS dotProduct,
#             sqrt(reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i]^2)) AS questionVectorNorm,
#             sqrt(reduce(s = 0.0, i in range(0, size(nodeVector)-1) | s + nodeVector[i]^2)) AS nodeVectorNorm
#         WITH n, dotProduct / (questionVectorNorm * nodeVectorNorm) AS cosineSimilarity
#         RETURN n, cosineSimilarity
#         ORDER BY cosineSimilarity DESC
#         LIMIT 10
#         """
#         result = session.run(query, question_vector=question_vector, top_n=top_n)
#         return [record for record in result]

In [4]:
# def query_vector_search(question_vector, top_n=5):
#     with driver.session() as session:
#         query = """
#         WITH $question_vector AS questionVector
#         MATCH (n)
#         WHERE n.namevector IS NOT NULL
#         WITH n, questionVector, n.namevector AS nodeVector
#         WITH n, questionVector, nodeVector,
#             reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i] * nodeVector[i]) AS dotProduct,
#             sqrt(reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i]^2)) AS questionVectorNorm,
#             sqrt(reduce(s = 0.0, i in range(0, size(nodeVector)-1) | s + nodeVector[i]^2)) AS nodeVectorNorm
#         WITH n, questionVector, dotProduct / (questionVectorNorm * nodeVectorNorm) AS cosineSimilarity
#         ORDER BY cosineSimilarity DESC
#         LIMIT 2
#         WITH collect({node: n, similarity: cosineSimilarity}) AS topNodes, questionVector
#         UNWIND topNodes AS topNodeWithSimilarity
#         WITH topNodeWithSimilarity.node AS topNode, topNodeWithSimilarity.similarity AS originalSimilarity, questionVector
#         MATCH (topNode)-[r]->(connectedNode)
#         WHERE connectedNode.vector IS NOT NULL
#         WITH topNode, connectedNode, type(r) AS relationshipType, originalSimilarity, questionVector
#         WITH topNode, connectedNode, relationshipType, originalSimilarity, connectedNode.vector AS connectedVector, questionVector
#         WITH topNode, connectedNode, relationshipType, originalSimilarity, connectedVector,
#             reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i] * connectedVector[i]) AS connectedDotProduct,
#             sqrt(reduce(s = 0.0, j in range(0, size(questionVector)-1) | s + questionVector[j]^2)) AS questionVectorNorm,
#             sqrt(reduce(s = 0.0, j in range(0, size(connectedVector)-1) | s + connectedVector[j]^2)) AS connectedVectorNorm
#         WITH topNode, connectedNode, relationshipType, originalSimilarity,
#             connectedDotProduct / (questionVectorNorm * connectedVectorNorm) AS connectedSimilarity
#         WITH topNode, connectedNode, relationshipType,
#             originalSimilarity + connectedSimilarity AS overallSimilarity
#         ORDER BY overallSimilarity DESC
#         LIMIT 15
#         RETURN topNode.description AS topNodeDescription, topNode.name AS topNodeName, connectedNode.description AS connectedNodeDescription, connectedNode.name AS connectedNodeName, relationshipType

#         """
#         result = session.run(query, question_vector=question_vector, top_n=top_n)
#         return [record for record in result]

In [5]:
question = "Can I call on Emergency, Satellite and Special Services Numbers using Wi-Fi Calling Services?"

In [223]:
def query_vector_search(question_vector, top_n=5):
    with driver.session() as session:
        query = """
        WITH $question_vector AS questionVector
        MATCH (n)
        WHERE n.namevector IS NOT NULL
        WITH n, questionVector, n.namevector AS nodeVector
        WITH n, questionVector, nodeVector,
            reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i] * nodeVector[i]) AS dotProduct,
            sqrt(reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i]^2)) AS questionVectorNorm,
            sqrt(reduce(s = 0.0, i in range(0, size(nodeVector)-1) | s + nodeVector[i]^2)) AS nodeVectorNorm
        WITH n, questionVector, dotProduct / (questionVectorNorm * nodeVectorNorm) AS cosineSimilarity
        ORDER BY cosineSimilarity DESC
        LIMIT 3
        WITH collect({node: n, similarity: cosineSimilarity}) AS topNodes, questionVector
        UNWIND topNodes AS topNodeWithSimilarity
        WITH topNodeWithSimilarity.node AS topNode, topNodeWithSimilarity.similarity AS originalSimilarity, questionVector
        MATCH (topNode)-[r]->(connectedNode)
        WHERE connectedNode.namevector IS NOT NULL AND r.vector IS NOT NULL
        WITH topNode, connectedNode, r, type(r) AS relationshipType, r.vector AS relVector, originalSimilarity, questionVector
        WITH topNode, connectedNode, relationshipType, relVector, originalSimilarity, connectedNode.namevector AS connectedVector, questionVector
        WITH topNode, connectedNode, relationshipType, relVector, originalSimilarity, connectedVector,
            reduce(s = 0.0, i in range(0, size(questionVector)-1) | s + questionVector[i] * connectedVector[i]) AS connectedDotProduct,
            sqrt(reduce(s = 0.0, j in range(0, size(questionVector)-1) | s + questionVector[j]^2)) AS questionVectorNorm,
            sqrt(reduce(s = 0.0, j in range(0, size(connectedVector)-1) | s + connectedVector[j]^2)) AS connectedVectorNorm,
            reduce(s = 0.0, k in range(0, size(questionVector)-1) | s + questionVector[k] * relVector[k]) AS relDotProduct,
            sqrt(reduce(s = 0.0, l in range(0, size(questionVector)-1) | s + questionVector[l]^2)) AS relationshipVectorNorm,
            sqrt(reduce(s = 0.0, m in range(0, size(relVector)-1) | s + relVector[m]^2)) AS relVectorNorm
        WITH topNode, connectedNode, relationshipType, originalSimilarity,
            connectedDotProduct / (questionVectorNorm * connectedVectorNorm) AS connectedSimilarity,
            relDotProduct / (relationshipVectorNorm * relVectorNorm) AS relationshipSimilarity
        WITH topNode, connectedNode, relationshipType,
            3/2*originalSimilarity + 1/2*connectedSimilarity + 1/2*relationshipSimilarity AS overallSimilarity
        ORDER BY overallSimilarity DESC
        LIMIT 10
        RETURN topNode.description AS topNodeDescription, topNode.name AS topNodeName, connectedNode.description AS connectedNodeDescription, connectedNode.name AS connectedNodeName, relationshipType
        """
        result = session.run(query, question_vector=question_vector, top_n=top_n)
        return [record for record in result]

In [224]:
question_vector = encode_model.encode(question)
response = query_vector_search(question_vector=question_vector.tolist())
response_text = []
para = ""
topNodeName = " "
i = 0
for record in response:
    if topNodeName != record["topNodeName"]:
        para += "HERE ENDS THE PREVIOUS CONNECTIONS. \n ------------------------\nNEW CONNECTIONS LIST STARTS \n"
        topNodeName = record["topNodeName"]
        para += "TOP NODE -> " + record["topNodeName"] + " : " + record["topNodeDescription"] + " THESE ARE THE CONNECTED NODES : " + "\n"
    para += "CONNECTED NODE -> " + record["connectedNodeName"] + ". Relation is " + record["relationshipType"] + " ." + " Description of Node Is " + record["connectedNodeDescription"] + "\n"
print("Context Found : \n", para)

<Record topNodeDescription='can be availed using Wi-Fi Calling pack only and will not be available for PayGo customers requires a stable Wi-Fi connection to prevent call drop issues does not support calls to Emergency, Satellite, or Special services numbers' topNodeName='Wi-Fi Calling Service' connectedNodeDescription='Roaming in 150+ countries with a single plan International roaming service Users can continue using International Roaming (IR) service on international networks by purchasing an IR pack. does not work on prepaid cards of Jammu & Kashmir, Assam, and North East when in International Roaming due to Indian Regulations It will work outside India on voice, SMS, Wi-Fi, and data if International Roaming is activated and a valid recharge plan is present on the Jio number.' connectedNodeName='International Roaming (service)' relationshipType='RelatedTo'>
<Record topNodeDescription='can be availed using Wi-Fi Calling pack only and will not be available for PayGo customers requires 

In [229]:
###################### MODEL FOR QA WITH CONTEXT ###########################

system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for question answering.
    Context from retrived document is provided to you. 
    Use that and general knowledge to answer user queries.
    Dont mention that context was provided
    """,
    input_variables = ["relationship_types"],
)


system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)


parser = JsonOutputParser()

human_prompt = PromptTemplate(
    template = """ From the following \nContext: {context} answer the following question \nText: {text}""",
    input_variables=["context", "text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt_qa = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_qa = chat_prompt_qa | model

In [230]:
response  = chain_qa.invoke({"relationship_types" : relation_types, "context": para, "text" : question})
print("Answer : ", response.content)

No, you cannot call on Emergency, Satellite, or Special Services numbers using Wi-Fi Calling Services.
